firestation_count: 구 내 소방서 및 안전센터 수

firestation_type_ratio: 소방서 대비 안전센터 비율 (보조 지표)

소방서는 광역적 관할, 안전센터/구조대는 생활권 대응 역할

firestation_density: 구 면적 대비 소방시설 밀도 (면적 데이터와 결합 시 활용)

👉 현재 데이터셋만으로는 주소 → 구 매핑 과정이 필요
(주소 컬럼이 없고, 서·센터명만 있으므로 행정구 매핑 테이블을 만들거나
X,Y 좌표를 행정구 경계 Shapefile과 spatial join 하면 정확히 매핑 가능)

In [1]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

In [2]:
# 1. 데이터 불러오기
df = pd.read_csv("../../data/FIR-FIRESTATION/FIR-FIRESTATION.csv")
print("원본 크기:", df.shape)
display(df.head())

# 2. 구 추출 (서·센터명에서 '구' 포함된 부분을 기준)
# 예: '강북소방서' → '강북구', '쌍문119안전센터' → '도봉구'
# (※ 더 정확히 하려면 행정동 좌표 기반 Spatial Join 필요)
def extract_gu(name):
    for gu in ["종로구","중구","용산구","성동구","광진구","동대문구","중랑구","성북구","강북구","도봉구",
               "노원구","은평구","서대문구","마포구","양천구","강서구","구로구","금천구","영등포구",
               "동작구","관악구","서초구","강남구","송파구","강동구"]:
        if gu.replace("구","") in name:
            return gu
    return None

df["gu"] = df["서ㆍ센터명"].apply(extract_gu)

# 3. 구별 집계
agg = df.groupby("gu").agg(
    firestation_count=("서ㆍ센터명","count"),
    firestation_type_unique=("유형구분명","nunique")
).reset_index()

# 소방서/센터 비율
agg["firestation_type_ratio"] = (
    df.groupby("gu")["유형구분명"].apply(lambda x: (x=="소방서").sum() / len(x))
).values

display(agg.head())

# 4. 점수화
scaler = MinMaxScaler(feature_range=(0,100))
scaled = scaler.fit_transform(agg[["firestation_count", "firestation_type_ratio"]])

score_df = agg[["gu"]].copy()
score_df["firestation_count_score"] = scaled[:,0]
score_df["firestation_ratio_score"] = scaled[:,1]

# 5. 종합 점수
score_df["firestation_mean_score"] = score_df.filter(like="_score").mean(axis=1)

# 6. 저장
score_df.to_csv("../../result/FIR/FIR-FIRESTATION_result.csv", index=False, encoding="utf-8-sig")

score_df.head()

원본 크기: (180, 9)


,연번,서ㆍ센터ID,서ㆍ센터명,유형구분명,도서지역포함여부,상위서ㆍ센터ID,일련번호,X좌표,Y좌표
0,1,1110108,쌍문119안전센터,안전센터/구조대,NaN,1110000,1,202367.981000,560953.287000
1,2,1125000,강북소방서,소방서,NaN,1100000,2,203372.134150,559271.253735
2,3,1125101,번동119안전센터,안전센터/구조대,NaN,1125000,3,203369.039207,559276.675604
3,4,1125201,구조대,안전센터/구조대,NaN,1125000,4,203375.742707,559265.794952
4,5,1113105,공덕119안전센터,안전센터/구조대,NaN,1113000,5,196444.779849,550061.600108


,gu,firestation_count,firestation_type_unique,firestation_type_ratio
0,강남구,1,1,1.000000
1,강동구,1,1,1.000000
2,강북구,1,1,1.000000
3,강서구,1,1,1.000000
4,관악구,3,2,0.333333


,gu,firestation_count_score,firestation_ratio_score,firestation_mean_score
0,강남구,0.0,100.000000,50.000000
1,강동구,0.0,100.000000,50.000000
2,강북구,0.0,100.000000,50.000000
3,강서구,0.0,100.000000,50.000000
4,관악구,50.0,16.666667,33.333333
